In [1]:
import pandas as pd
from nltk import FreqDist
import pickle
import numpy as np
from gensim.models import Word2Vec

In [2]:
import numpy as np
import pandas as pd
import os
import requests
import re
from nltk import FreqDist
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
import pickle
from cltk.stops.grc import STOPS as stopwords
from gensim.corpora import Dictionary
import unicodedata
import json

In [3]:
LAGT = pd.read_json("../data/large_data/LAGT_metadata.json")

In [4]:
LAGT.head(5)

,author_id,doc_id,GLAUx?,filename,subcorpus,author,title,date_avr,provenience,tlg_epithet,n_sentences,wordcount
0,tlg0001,tlg0001.tlg001,True,tlg0001.tlg001.perseus-grc2.xml,classical,Apollonius Rhodius,Argonautica,-2.5,pagan,Epici/-ae,3252,38822
1,tlg0003,tlg0003.tlg001,True,tlg0003.tlg001.perseus-grc2.xml,classical,Thucydides,The Peloponnesian War,-4.5,pagan,Historici/-ae,6068,150118
2,tlg0004,tlg0004.tlg001,False,tlg0004.tlg001.perseus-grc1.xml,roman,Diogenes Laertius,Lives of Eminent Philosophers,2.5,pagan,Biographi,10245,110763
3,tlg0005,tlg0005.tlg001,True,tlg0005.tlg001.perseus-grc1.xml,classical,Theocritus,Idylls,-3.0,pagan,Bucolici,1982,19200
4,tlg0005,tlg0005.tlg002,True,tlg0005.tlg002.perseus-grc1.xml,classical,Theocritus,Epigrams,-3.0,pagan,Bucolici,152,1734


In [5]:
subcorpora = ['archaic', 'classical', 'roman', 'christian', 'jewish']

# Read subcorpora from ngrams - developing a memory-friendly reader

In [6]:
ids_lines = pickle.load(open("../data/ids_lines.pickle", "rb"))

In [7]:
list(ids_lines.items())[:5]

[('tlg0001.tlg001', (0, 41193)),
 ('tlg0003.tlg001', (41193, 184616)),
 ('tlg0004.tlg001', (184616, 280036)),
 ('tlg0005.tlg001', (280036, 300693)),
 ('tlg0005.tlg002', (300693, 302641))]

In [8]:
subcorpus_ids = LAGT[LAGT["subcorpus"]=="archaic"]["doc_id"].tolist()
len(subcorpus_ids)

26

In [9]:
lines_list = [ ]
for id in subcorpus_ids:
    lines_list.extend([l for l in range(ids_lines[id][0], ids_lines[id][1])])
ngrams_n = len(lines_list)
ngrams_n

339423

In [10]:
sample_size=1000000
if sample_size != None:
    sample_inx =np.random.randint(0, ngrams_n, sample_size)
else:
    sample_inx =[n for n in range(0, ngrams_n)]

In [11]:
sample_lines = np.array(lines_list)[sample_inx]
len(sample_lines)

1000000

In [12]:
sample_lines[:5]

array([7902850, 2075737, 8978287, 1931259, 3268909])

In [13]:
test_ngrams = []
with open("../data/large_data/corpus_ngrams_bydocid.txt", "r") as f:
    lines = f.readlines()
    for li in sample_lines:
        line = lines[li]# for line index in subcorpus lineindexes
        if bool(line.split()):
            test_ngrams.append(line.split())

In [14]:
len(test_ngrams)

999651

# Sample corpora with memory-friendly reader

In [15]:
class NgramCorpusSample(object):
    def __init__(self, ids_list, ids_lines, fname, sample_size=None, sample_seed=1, bow=False, dct=None):
        self.ids_list = ids_list
        self.fname = fname
        self.bow  = bow
        self.dct = dct
        self.sample_size = sample_size
        self.ids_lines = ids_lines
        lines_list = [ ]
        for id in ids_list:
            lines_list.extend([l for l in range(ids_lines[id][0], ids_lines[id][1])])
        ngrams_n = len(lines_list)
        if sample_size != None:
            np.random.seed(sample_seed)
            sample_inx =np.random.randint(0, ngrams_n, sample_size)
        else:
            sample_inx =[n for n in range(0, ngrams_n)]
        sample_lines = np.array(lines_list)[sample_inx]
        self.len = len(sample_lines)
        self.sample_lines = sample_lines
    def __len__(self):
        #[el for el in self]
        return self.len
    def __iter__(self):
        with open(self.fname, "r") as f:
            lines = f.readlines()
            for li in self.sample_lines:
                line = lines[li] # for line index in subcorpus lineindexes
                if bool(line.split()):
                    if (self.bow) & (self.dct != None):
                        yield self.dct.doc2bow(line.split())
                    else:
                        yield line.split()

In [16]:
# test with classical

In [17]:
subcorpus_ids = LAGT[LAGT["subcorpus"]=="roman"]["doc_id"].tolist()
len(subcorpus_ids)

596

In [18]:
subcorpus_ngrams =  NgramCorpusSample(subcorpus_ids, ids_lines, "../data/large_data/corpus_ngrams_bydocid.txt", sample_size=1000000)

In [19]:
len(subcorpus_ngrams)

1000000

In [20]:
len([doc for doc in subcorpus_ngrams])

980208

In [21]:
FreqDist([w for ng in subcorpus_ngrams for w in ng]).most_common()[:50]

[('εἰμί', 86416),
 ('αὐτός', 68360),
 ('οὗτος', 55211),
 ('λέγω', 37546),
 ('γίγνομαι', 31525),
 ('τις', 30334),
 ('ἔχω', 29557),
 ('πολύς', 28087),
 ('πᾶς', 22650),
 ('ἄλλος', 21668),
 ('φημί', 17161),
 ('ποιέω', 15230),
 ('λόγος', 14224),
 ('ἐκεῖνος', 11745),
 ('τοιοῦτος', 11694),
 ('μέγας', 11540),
 ('πρῶτος', 10766),
 ('οὐδείς', 9887),
 ('ἕτερος', 9696),
 ('πόλις', 9384),
 ('σῶμα', 8978),
 ('ἅπας', 8360),
 ('ἀγαθός', 8282),
 ('φύσις', 8149),
 ('δύναμις', 8130),
 ('μέρος', 7951),
 ('ἄνθρωπος', 7917),
 ('εἷς', 7831),
 ('ἀρχή', 7814),
 ('λαμβάνω', 7582),
 ('ἕκαστος', 7256),
 ('δοκέω', 7100),
 ('ὁράω', 6997),
 ('δέω', 6949),
 ('μόνος', 6823),
 ('ἀνήρ', 6793),
 ('καλέω', 6710),
 ('δύναμαι', 6256),
 ('ὑπάρχω', 6136),
 ('χρόνος', 5940),
 ('θεός', 5916),
 ('χράω', 5609),
 ('δίδωμι', 5491),
 ('ὅλος', 4946),
 ('ἡμέρα', 4905),
 ('μηδείς', 4883),
 ('γράφω', 4851),
 ('γένος', 4826),
 ('τὶς', 4718),
 ('τίς', 4716)]

In [22]:
with open("../data/large_data/subcorpora_vocabs.pickle", "rb") as f:
    subcorpora_vocabs = pickle.load(f)

In [23]:
subcorpus_ngrams_seed1 =  NgramCorpusSample(subcorpus_ids, ids_lines, "../data/large_data/corpus_ngrams_bydocid.txt", sample_size=1000000, sample_seed=1)
subcorpus_ngrams_seed2 =  NgramCorpusSample(subcorpus_ids, ids_lines, "../data/large_data/corpus_ngrams_bydocid.txt", sample_size=1000000, sample_seed=2)

In [24]:
freqs_comparison = pd.DataFrame(
    [dict(subcorpora_vocabs["roman"]["word_freqs"][:5000]),
    dict(FreqDist([w for ng in subcorpus_ngrams_seed1 for w in ng]).most_common()[:5000]),
    dict(FreqDist([w for ng in subcorpus_ngrams_seed2 for w in ng]).most_common()[:5000])]
).T

In [25]:
freqs_comparison.dropna().corr()

,0,1,2
0,1.000000,0.998884,0.998790
1,0.998884,1.000000,0.999912
2,0.998790,0.999912,1.000000


In [26]:
len(freqs_comparison)

5365

In [27]:
freqs_comparison[:4000].corr(method="spearman")

,0,1,2
0,1.000000,0.987949,0.988137
1,0.987949,1.000000,0.985046
2,0.988137,0.985046,1.000000


In [28]:
np.log2(freqs_comparison).corr(method="spearman")

,0,1,2
0,1.000000,0.986517,0.986683
1,0.986517,1.000000,0.982736
2,0.986683,0.982736,1.000000


In [29]:
# let's build two models based on the same subcorpus but sampled with different seed

In [30]:
%%time
sub = "classical"
subcorpus_ids = LAGT[LAGT["subcorpus"]=="roman"]["doc_id"].tolist()

subcorpus_ngrams_seed1 =  NgramCorpusSample(subcorpus_ids, ids_lines, "../data/large_data/corpus_ngrams_bydocid.txt", sample_size=1000000, sample_seed=1)
model_seed1 = Word2Vec(vector_size=150, window=3, negative=5, ns_exponent=1, sg=0, epochs=5, workers=8)
model_seed1.build_vocab_from_freq(word_freq=dict(subcorpora_vocabs[sub]["word_freqs"][:5000]))
model_seed1.train(subcorpus_ngrams, total_examples=len(subcorpus_ngrams_seed1), epochs=model_seed1.epochs)
vectors_seed1 = model_seed1.wv

subcorpus_ngrams_seed2 =  NgramCorpusSample(subcorpus_ids, ids_lines, "../data/large_data/corpus_ngrams_bydocid.txt", sample_size=1000000, sample_seed=2)
model_seed2 = Word2Vec(vector_size=150, window=3, negative=5, ns_exponent=1, sg=0, epochs=5, workers=8)
model_seed2.build_vocab_from_freq(word_freq=dict(subcorpora_vocabs[sub]["word_freqs"][:5000]))
model_seed2.train(subcorpus_ngrams, total_examples=len(subcorpus_ngrams_seed2), epochs=model_seed1.epochs)
vectors_seed2 = model_seed2.wv

CPU times: user 1min 29s, sys: 1min 40s, total: 3min 9s
Wall time: 3min 40s


In [31]:
vectors_seed1.most_similar("θεός", topn=20)

[('θυσία', 0.7014735341072083),
 ('εὔχομαι', 0.644244372844696),
 ('θεῖος', 0.6418712139129639),
 ('ἥρως', 0.6404505968093872),
 ('θύω', 0.6360121965408325),
 ('δαίμων', 0.6353362798690796),
 ('σέβω', 0.6338143944740295),
 ('Ζεύς', 0.6313347816467285),
 ('σωτήρ', 0.6171830296516418),
 ('ὅσιος', 0.6013766527175903),
 ('τιμάω', 0.5994257926940918),
 ('ἱερόν', 0.5989795923233032),
 ('εὐσεβής', 0.5937182307243347),
 ('εὐχή', 0.5934133529663086),
 ('ὑμνέω', 0.5852774381637573),
 ('Ἀπόλλων', 0.5815355181694031),
 ('Ἡρακλῆς', 0.576744556427002),
 ('πάτριος', 0.5744653940200806),
 ('εὐεργέτης', 0.5719175934791565),
 ('προσκυνέω', 0.56976318359375)]

In [32]:
vectors_seed2.most_similar("θεός", topn=20)

[('θυσία', 0.7163611650466919),
 ('σέβω', 0.6574140787124634),
 ('θεῖος', 0.6565834283828735),
 ('εὔχομαι', 0.6547293663024902),
 ('δαίμων', 0.6413161158561707),
 ('θύω', 0.6328307390213013),
 ('ὅσιος', 0.6319435238838196),
 ('Ζεύς', 0.6233252882957458),
 ('σωτήρ', 0.6160833835601807),
 ('ἥρως', 0.615529477596283),
 ('τιμάω', 0.5929490923881531),
 ('Ἀπόλλων', 0.5900370478630066),
 ('ὑμνέω', 0.5896173715591431),
 ('πάτριος', 0.5870264768600464),
 ('εὐσεβής', 0.5869643092155457),
 ('εὐχή', 0.5867000818252563),
 ('εὐδαίμων', 0.5801908373832703),
 ('θεά', 0.5786188840866089),
 ('δαιμόνιον', 0.5773245692253113),
 ('βροτός', 0.5772062540054321)]

In [33]:
for sub in subcorpora:
    subcorpus_ids = LAGT[LAGT["subcorpus"]==sub]["doc_id"].tolist()
    #do the stuff above
    print("generating model for the {} subcorpus...".format(sub))
    subcorpus_ngrams_seed1 =  NgramCorpusSample(subcorpus_ids, ids_lines, "../data/large_data/corpus_ngrams_bydocid.txt", sample_size=1000000, sample_seed=1)
    model_seed1 = Word2Vec(vector_size=150, window=3, negative=5, ns_exponent=1, sg=0, epochs=5, workers=8)
    model_seed1.build_vocab_from_freq(word_freq=dict(subcorpora_vocabs[sub]["word_freqs"][:5000]))
    model_seed1.train(subcorpus_ngrams, total_examples=len(subcorpus_ngrams_seed1), epochs=model_seed1.epochs)
    vectors_seed1 = model_seed1.wv
    vectors_seed1.save("../data/large_data/vectors_{}_seed1.kv".format(sub))
    subcorpus_ngrams_seed2 =  NgramCorpusSample(subcorpus_ids, ids_lines, "../data/large_data/corpus_ngrams_bydocid.txt", sample_size=1000000, sample_seed=2)
    model_seed2 = Word2Vec(vector_size=150, window=3, negative=5, ns_exponent=1, sg=0, epochs=5, workers=8)
    model_seed2.build_vocab_from_freq(word_freq=dict(subcorpora_vocabs[sub]["word_freqs"][:5000]))
    model_seed2.train(subcorpus_ngrams, total_examples=len(subcorpus_ngrams_seed2), epochs=model_seed1.epochs)
    vectors_seed2 = model_seed2.wv
    vectors_seed2.save("../data/large_data/vectors_{}_seed2.kv".format(sub))
    print("{} model ready".format(sub))


generating model for the archaic subcorpus...
archaic model ready
generating model for the classical subcorpus...
classical model ready
generating model for the roman subcorpus...
roman model ready
generating model for the christian subcorpus...
christian model ready
generating model for the jewish subcorpus...
jewish model ready
